In [30]:
import numpy as np
import cv2 as cv
import keyboard # nincs benne az alap anacondában lehet instalálni kell - pip install keyboard
from time import time

#---------------------------------------- Jelfeldolgozáshoz -------------------------------------

Morsekod = { 'A':'.-', 'B':'-...', 
             'C':'-.-.', 'D':'-..', 'E':'.', 
             'F':'..-.', 'G':'--.', 'H':'....', 
             'I':'..', 'J':'.---', 'K':'-.-', 
             'L':'.-..', 'M':'--', 'N':'-.', 
             'O':'---', 'P':'.--.', 'Q':'--.-', 
             'R':'.-.', 'S':'...', 'T':'-', 
             'U':'..-', 'V':'...-', 'W':'.--', 
             'X':'-..-', 'Y':'-.--', 'Z':'--..', 
             '1':'.----', '2':'..---', '3':'...--', 
             '4':'....-', '5':'.....', '6':'-....', 
             '7':'--...', '8':'---..', '9':'----.', 
             '0':'-----', ', ':'--..--', '.':'.-.-.-', 
             '?':'..--..', '/':'-..-.', '-':'-....-', 
             '(':'-.--.', ')':'-.--.-'}

def fordito(uzenet):
    if(uzenet == ''):
        return ''
    #uzenet += ' ' # hozzá kell adni egy space-t az üzenethez, hogy a végén ki tudjon lépni a for ciklusból
    kimenet = ''
    betu = '' # egy-egy betű kódját tároljuk ebben
    space = 0 # a space-k számlálója

    for karakter in uzenet: # végigmegy egyenként az összes karakteren
        
        if (karakter != ' '): 
            betu += karakter # ha nem space van akkor az egy betű része, ezért le kell tárolni, hogy 
            space = 0 # lenullázzuk a számlálót
        else:
            space += 1 # space esetén hozzáadunk egyet a számlálóhoz
            if (space == 1): # ha egy space van, akkor az egy betűnek a vége
                if (betu in list(Morsekod.values())):
                    kimenet += list(Morsekod.keys())[list(Morsekod.values()).index(betu)] #kiolvassuk a betűt a listából
                else:
                    kimenet += '?'
                betu = '' # nullázzuk a betű változót
            elif (space == 2): # a másik eset, ha ezelőtt már volt egy space, ekkor agy szó végéhez értünk
                kimenet += ' ' # a kimenetben a szavakat elválasztó space
                
    return kimenet

#------------------------------------ Képfeldolgozás --------------------------------------

def feny():
    ret, frame = cap.read()
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    ret2, tr_frame = cv.threshold(frame,253,255,cv.THRESH_BINARY)
    subtracted_frame=tr_frame-base_frame # alapkép kivonása, hogy az alapból benne levő fényforrások ne legyenek jelen mint zaj
    kernel = np.ones((5,5),np.uint8) # kernel az erode-hoz
    eroded_f_frame = cv.erode(subtracted_frame,kernel,iterations = 1) #kép erode-álása, tehát az egy-egy maradék zajpixel kivonása
    cv.imshow('frame',frame)
    cv.imshow('szurke',eroded_f_frame)
    van_e_feny = sum(sum(eroded_f_frame))>10*255
    return(van_e_feny)

#----------------------------------- További definíciók -----------------------------------

hibahatar = 0.5 # A jelek hosszának hibahatára (max 0.5)
egyseg = 1 # A jelek alapegysége (a rövid jel hossza)

def hibahataron_belul(jelhossz, vart_jel): # Ellenőrzi, hogy adott hibahatár mellett a mért idő egy adott jelnek tekinthető-e
    if (jelhossz > (vart_jel - vart_jel*hibahatar) and jelhossz < (vart_jel + vart_jel*hibahatar)):
        return True
    return False

#-------------------------------------- A főprogram ---------------------------------------

# üres változók tároláshoz
karakterek = ''
betuk = ''
szoveg_ford = '' # ha majd implementáljuk a fordítót akkor talán érdemes külön eltárolni és nem módosítani de ez csak javaslat
szoveg = ''

cap = cv.VideoCapture(0) # kamera
ret, frame = cap.read() # kamera képe
gray_base_frame= cv.cvtColor(frame, cv.COLOR_BGR2GRAY) # alapkép szürkébe
ret, base_frame = cv.threshold(gray_base_frame,253,255,cv.THRESH_BINARY) # alapkép thresholdolva
van_feny = False # előző állapot tárolására
start = 0 # időmérés kezdete
end = 0 # időmérés vége
varakozas = True # várakozó állapothoz

while(1):
    if (feny()):
        if (van_feny == False): # Legutóbb nem volt fény, tehát éppen állapotváltozás történt
            
            varakozas = False
            jelhossz = end - start # idők különbségei

            if (hibahataron_belul(jelhossz, 3*egyseg)): # Betűköz
                betuk += karakterek + ' ' # az eddigi morse karaktereket beteszi egy betűk végére, a szoköz a fordítóban a betűközt jelenti
                karakterek = '' 

            elif (hibahataron_belul(jelhossz, 7*egyseg)): # Szóköz/Mondatköz
                betuk += karakterek + '  ' # az eddigi morse karaktereket beteszi egy betűk végére, a két szoköz a fordítóban a szóközt jelenti
                szoveg += betuk # a morze szöveghez hozzáadja a legutóbbi morze betűt
                karakterek = '' 
                szoveg_ford += fordito(betuk) # a fordított szöveghez hozzáadja az imént dekódolt szót
                print(fordito(betuk)) # Dekódolt szó kiírása
                betuk = ''

            
            start = time() # Idő fehér kezdetekor
            van_feny = True # Átállítjuk a jelzőt
        
        end = time()  # Folyamatosan frissítjük az end-et, amíg vége nem lesz a jelnek
   
    elif(varakozas == False): # hosszú sötétség esetén felesleges számításokat nem végzünk (várakozó állapot)
        if (van_feny == True): # Legutobb volt fény, tehát éppen állapotváltozás történt
            
            jelhossz = end - start #idők különbségei

            if (hibahataron_belul(jelhossz, egyseg)): # rövid karakter
                karakterek += '.'

            elif (hibahataron_belul(jelhossz, 3*egyseg)): # hosszú karakter
                karakterek += '-'

            start = time() # Idő fekete kezdetekor
            van_feny = False # Átállítjuk a jelzőt

        end = time() # Folyamatosan frissítjük az end-et, amíg vége nem lesz a szünetnek
        
        if ((end - start) > (7*egyseg + 7*egyseg*hibahatar)):
            betuk += karakterek + '  ' # az eddigi morse karaktereket beteszi egy betűk végére, a két szoköz a fordítóban a szóközt jelenti
            szoveg += betuk # a morze szöveghez hozzáadja a legutóbbi morze betűt
            karakterek = '' 
            szoveg_ford += fordito(betuk)# a fordított szöveghez hozzáadja az imént dekódolt szót
            print(fordito(betuk)) # Dekódolt szó kiírása
            betuk = ''
            
            # időmérő változók alaphelyzetbe állítása, várakozó módra váltás
            end = 0
            start = 0
            varakozas = True
    
    # esc lenyomására leáll a program
    if cv.waitKey(1) & 0xFF == 27:
        break
    
print(szoveg_ford)
print(szoveg)

cap.release()
cv.destroyAllWindows()

SOS 
SEGITSEG 
SOS SEGITSEG 
... --- ...  ... . --. .. - ... . --.  
